In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
import warnings # 0 warnings :)
warnings.filterwarnings("ignore")
from matplotlib.pyplot import figure
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
import numpy as np

### Set to True to update all the pics in the /Figure folder
downloadPic = False
# to-use:
# if (downloadPic):
#     plt.savefig ('Figure/nameOfPic.png')

In [2]:
df = pd.read_csv('electricity.csv')
df.drop_duplicates()
# drop rows containing na values
df = df.dropna()
df['Utility.State'] = df['Utility.State'].astype('category')
df['Utility.Type'] = df['Utility.Type'].astype('category')
df = df.rename(columns={'Demand.Summer Peak': 'Demand.SummerPeak', 'Demand.Winter Peak': 'Demand.WinterPeak', 'Uses.No Charge':'Uses.NoCharge'})
df.head(5)

,Utility.Number,Utility.Name,Utility.State,Utility.Type,Demand.SummerPeak,Demand.WinterPeak,Sources.Generation,Sources.Purchased,Sources.Other,Sources.Total,...,Retail.Commercial.Customers,Retail.Industrial.Revenue,Retail.Industrial.Sales,Retail.Industrial.Customers,Retail.Transportation.Revenue,Retail.Transportation.Sales,Retail.Transportation.Customers,Retail.Total.Revenue,Retail.Total.Sales,Retail.Total.Customers
0,55,City of Aberdeen - (MS),MS,Municipal,32.7,30.7,0.0,204875.0,0.0,204875.0,...,661.0,6025.0,118989.0,2.0,0.0,0.0,0.0,14649.0,198442.0,3258.0
1,59,City of Abbeville - (LA),LA,Municipal,31.4,22.9,0.0,137537.0,0.0,137537.0,...,1032.0,3211.8,37566.0,27.0,0.0,0.0,0.0,12574.4,127849.0,5523.0
2,84,A & N Electric Coop,VA,Cooperative,152.6,156.1,19.0,714961.0,0.0,714980.0,...,4019.0,13952.7,169301.0,8.0,0.0,0.0,0.0,68496.7,672572.0,35610.0
3,97,Adams Electric Coop,IL,Cooperative,37.0,37.0,4677.0,170432.0,0.0,175109.0,...,222.0,0.0,0.0,0.0,0.0,0.0,0.0,22070.0,164358.0,8825.0
4,108,Adams-Columbia Electric Coop,WI,Cooperative,109.1,83.4,0.0,524637.0,0.0,524637.0,...,2074.0,15398.0,160275.0,926.0,0.0,0.0,0.0,67563.0,493241.0,37138.0


In [3]:
df = df[['Sources.Total', 'Revenue.Total', 'Retail.Total.Revenue', 'Retail.Total.Sales', 'Retail.Total.Customers']]

df.head(5)

,Sources.Total,Revenue.Total,Retail.Total.Revenue,Retail.Total.Sales,Retail.Total.Customers
0,204875.0,14949.5,14649.0,198442.0,3258.0
1,137537.0,12574.4,12574.4,127849.0,5523.0
2,714980.0,69505.2,68496.7,672572.0,35610.0
3,175109.0,22311.0,22070.0,164358.0,8825.0
4,524637.0,67566.0,67563.0,493241.0,37138.0


In [4]:
(df_train,df_test) = train_test_split(df, train_size=0.8, test_size=0.2, random_state=0)
features = df.drop('Revenue.Total', axis = 1)
features_train = df_train.drop('Revenue.Total',axis=1)
features_test = df_test.drop('Revenue.Total',axis=1)
targets_train = df_train['Revenue.Total']
targets_test = df_test['Revenue.Total']

stnd = StandardScaler()
features_train = pd.get_dummies(features_train)

stnd.fit(features_train)
features_train = pd.DataFrame(stnd.transform(features_train),
                              columns=features_train.columns) 
features_test = pd.DataFrame(stnd.transform(features_test),
                             columns=features_test.columns)

In [5]:
lr = LinearRegression()
lr.fit(features_train, targets_train)
print('training R2:', lr.score(features_train, targets_train))
print('test R2:', lr.score(features_test, targets_test))

training R2: 0.9593396359542248
test R2: 0.9811253055260128
